# Online GP regression with thomson sampling

In [39]:
import os
import time
from contextlib import ExitStack

import gpytorch
import gpytorch.settings as gpts
import torch
from gpytorch.constraints import Interval
from gpytorch.distributions import MultivariateNormal
from gpytorch.kernels import MaternKernel, RFFKernel, ScaleKernel
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.mlls import ExactMarginalLogLikelihood
from torch.quasirandom import SobolEngine

from botorch.fit import fit_gpytorch_mll
from botorch.generation import MaxPosteriorSampling
from botorch.models import SingleTaskGP
from botorch.test_functions import Hartmann
from botorch.utils.transforms import unnormalize

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.double
SMOKE_TEST = os.environ.get("SMOKE_TEST")

In [40]:
hart6 = Hartmann(dim=3, negate=True).to(device=device, dtype=dtype)
dim = hart6.dim


def eval_objective(x):
    """This is a helper function we use to unnormalize and evalaute a point"""
    return hart6(unnormalize(x, hart6.bounds))

In [41]:
def get_initial_points(dim, n_pts, seed=None):
    sobol = SobolEngine(dimension=dim, scramble=True, seed=seed)
    X_init = sobol.draw(n=n_pts).to(dtype=dtype, device=device)
    return X_init

In [42]:
def generate_batch(
    X,
    Y,
    batch_size,
    n_candidates,
    sampler="cholesky",  # "cholesky", "ciq", "rff"
    use_keops=False,
):
    assert sampler in ("cholesky", "ciq", "rff", "lanczos")
    assert X.min() >= 0.0 and X.max() <= 1.0 and torch.all(torch.isfinite(Y))

    # NOTE: We probably want to pass in the default priors in SingleTaskGP here later
    kernel_kwargs = {"nu": 2.5, "ard_num_dims": X.shape[-1]}
    if sampler == "rff":
        base_kernel = RFFKernel(**kernel_kwargs, num_samples=1024)
    else:
        base_kernel = (
            MaternKernel(**kernel_kwargs)
            if use_keops
            else MaternKernel(**kernel_kwargs)
        )
    covar_module = ScaleKernel(base_kernel)

    # Fit a GP model
    train_Y = (Y - Y.mean()) / Y.std()
    likelihood = GaussianLikelihood(noise_constraint=Interval(1e-8, 1e-3))
    model = SingleTaskGP(X, train_Y, likelihood=likelihood, covar_module=covar_module)
    mll = ExactMarginalLogLikelihood(model.likelihood, model)
    fit_gpytorch_mll(mll)

    # Draw samples on a Sobol sequence
    sobol = SobolEngine(X.shape[-1], scramble=True)
    X_cand = sobol.draw(n_candidates).to(dtype=dtype, device=device)

    # Thompson sample
    with ExitStack() as es:
        if sampler == "cholesky":
            es.enter_context(gpts.max_cholesky_size(float("inf")))
        elif sampler == "ciq":
            es.enter_context(gpts.fast_computations(covar_root_decomposition=True))
            es.enter_context(gpts.max_cholesky_size(0))
            es.enter_context(gpts.ciq_samples(True))
            es.enter_context(
                gpts.minres_tolerance(2e-3)
            )  # Controls accuracy and runtime
            es.enter_context(gpts.num_contour_quadrature(15))
        elif sampler == "lanczos":
            es.enter_context(
                gpts.fast_computations(
                    covar_root_decomposition=True, log_prob=True, solves=True
                )
            )
            es.enter_context(gpts.max_lanczos_quadrature_iterations(10))
            es.enter_context(gpts.max_cholesky_size(0))
            es.enter_context(gpts.ciq_samples(False))
        elif sampler == "rff":
            es.enter_context(gpts.fast_computations(covar_root_decomposition=True))

    with torch.no_grad():
        thompson_sampling = MaxPosteriorSampling(model=model, replacement=False)
        X_next = thompson_sampling(X_cand, num_samples=batch_size)

    return X_next

In [43]:
def run_optimization(
    sampler, n_candidates, n_init, max_evals, batch_size, use_keops=False, seed=None
):
    X = get_initial_points(dim, n_init, seed)
    
    print(X.shape)
    
    
    Y = torch.tensor(
        [eval_objective(x) for x in X], dtype=dtype, device=device
    ).unsqueeze(-1)
    print(f"{len(X)}) Best value: {Y.max().item():.2e}")

    while len(X) < max_evals:
        # Create a batch
        start = time.monotonic()
        X_next = generate_batch(
            X=X,
            Y=Y,
            batch_size=min(batch_size, max_evals - len(X)),
            n_candidates=n_candidates,
            sampler=sampler,
            use_keops=use_keops,
        )
        end = time.monotonic()
        print(f"Generated batch in {end - start:.1f} seconds")
        Y_next = torch.tensor(
            [eval_objective(x) for x in X_next], dtype=dtype, device=device
        ).unsqueeze(-1)

        # Append data
        X = torch.cat((X, X_next), dim=0)
        Y = torch.cat((Y, Y_next), dim=0)

        print(f"{len(X)}) Best value: {Y.max().item():.2e}")
    return X, Y

In [44]:
batch_size = 5
n_init = 10
max_evals = 50
seed = 12345  # To get the same Sobol points

shared_args = {
    "n_init": n_init,
    "max_evals": max_evals,
    "batch_size": batch_size,
    "seed": seed,
}

In [45]:
# This tutorial will run much faster if KeOps + a GPU is used
USE_KEOPS = False

if USE_KEOPS:
    import pykeops
    from gpytorch.kernels.keops import MaternKernel as KMaternKernel

N_CAND = 10_000
if USE_KEOPS:
    N_CAND = 50_000
if SMOKE_TEST:
    N_CAND = 10
    
N_CAND_CHOL = 10_000 if not SMOKE_TEST else 10

In [46]:
%load_ext memory_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


### Cholesky

Exact GP inference, is in $O(N^3)$ where $N$ is the size of the dataset.

In [47]:
%memit X_chol, Y_chol = run_optimization("cholesky", N_CAND_CHOL, **shared_args)

torch.Size([10, 3])
10) Best value: 2.89e+00
Generated batch in 0.8 seconds
15) Best value: 2.89e+00
Generated batch in 0.6 seconds
20) Best value: 3.78e+00
Generated batch in 0.6 seconds
25) Best value: 3.85e+00
Generated batch in 0.6 seconds
30) Best value: 3.85e+00
Generated batch in 0.5 seconds
35) Best value: 3.85e+00
Generated batch in 0.5 seconds
40) Best value: 3.86e+00
Generated batch in 0.5 seconds
45) Best value: 3.86e+00
Generated batch in 0.6 seconds
50) Best value: 3.86e+00
peak memory: 4898.50 MiB, increment: 0.03 MiB


# Random Fourier Features

Approximate exact inference. Is in $O(KN)$ where $K$ is the number of Fourier features and $N$ is the size of the dataset.

In [48]:
%memit X_rff, Y_rff = run_optimization("rff", N_CAND, **shared_args)

torch.Size([10, 3])
10) Best value: 2.89e+00
Generated batch in 0.2 seconds
15) Best value: 3.11e+00
Generated batch in 0.3 seconds
20) Best value: 3.84e+00
Generated batch in 0.3 seconds
25) Best value: 3.85e+00
Generated batch in 0.2 seconds
30) Best value: 3.86e+00
Generated batch in 0.2 seconds
35) Best value: 3.86e+00
Generated batch in 0.2 seconds
40) Best value: 3.86e+00
Generated batch in 0.2 seconds
45) Best value: 3.86e+00
Generated batch in 0.2 seconds
50) Best value: 3.86e+00
peak memory: 4898.51 MiB, increment: 0.01 MiB
